In [18]:
# pip install accelerate -U

In [19]:
# pip install transformers -U

In [20]:
# pip install transformers[torch]

In [21]:
# pip install datasets transformers

In [22]:
# pip install numpy

In [31]:
# pip install pandas

In [32]:
# pip install tensorflow

In [33]:
# pip install torch

In [34]:
# pip install cv2

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import os
import cv2

In [2]:
from transformers import ViTImageProcessor

# model_name_or_path = 'google/vit-base-patch16-224'
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

In [3]:
# import os

In [3]:
test_path = ("4765063/test")
train_path = ("4765063/train")
validation_path = ("4765063/validation")

In [4]:
def get_images(path):
    image_path = []
    struck =[]
    for i in os.listdir(path):
        for j in os.listdir(path+"/"+i):
            if j == ".ipynb_checkpoints":
              continue
            image_path.append(path+"/"+i+"/"+j)
            if i == "struck":
                struck.append(1)
            else:
                struck.append(0)
    return image_path , struck

In [5]:
test_image_path,test_struck = get_images(test_path)
train_image_path,train_struck = get_images(train_path)
validation_image_path,validation_struck = get_images(validation_path)


train_image_path = test_image_path+train_image_path
train_struck = test_struck+train_struck

In [6]:
type(test_struck[0])

int

In [8]:
def image_preprocessing(image_path):
    images=[]
    for i in image_path:
        # print(i)
        img = cv2.imread(i)
        # converting into grayscale
        gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # convrting into binaryimage
        _, binary_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
        binary_image = cv2.resize(binary_image, (224, 224))
        # binary_image = np.expand_dims(binary_image, axis=-1)
        binary_image = cv2.merge([binary_image, binary_image, binary_image])
        binary_image = binary_image/255
        binary_image = torch.from_numpy(binary_image)
        images.append(binary_image)
    return images

In [9]:
train_images = image_preprocessing(train_image_path)
train_images = torch.stack(train_images)
train_images = train_images.permute(0, 3, 1, 2)
# train_images = image_preprocessing(train_image_path[:10])
validation_images = image_preprocessing(validation_image_path)
validation_images = torch.stack(validation_images)
validation_images = validation_images.permute(0, 3, 1, 2)

In [10]:
len(train_images), len(train_struck),len(validation_images),len(validation_struck)

(1134, 1134, 252, 252)

In [11]:
# train = pd.DataFrame({'x': train_images, 'y':train_struck })
# validation = pd.DataFrame({'x': validation_images, 'y':validation_struck })

train = {
    'image' : train_images,
    'label' :train_struck
}

validation = {
    'image' : validation_images,
    'label' : validation_struck
}

from datasets import Dataset
train_dataset = Dataset.from_dict(train)
validation_dataset = Dataset.from_dict(validation)


In [12]:
import torch

# def collate_fn(batch):
#     pixel_values = torch.stack([item['pixel_values'] for item in batch])
#     labels = torch.tensor([item['labels'] for item in batch])
#     return {'pixel_values': pixel_values, 'labels': labels}

def collate_fn(batch):
    pixel_values = torch.stack([torch.from_numpy(np.array(item['image'])) for item in batch])
    # pixel_values = torch.stack([torch.from_numpy(item) for item in batch['x']])
    labels = torch.tensor([item['label'] for item in batch])

    return {'pixel_values': pixel_values, 'labels': labels}


In [13]:
from datasets import load_metric

metric = load_metric("precision")

def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    references = p.label_ids
    return metric.compute(predictions=predictions, references=references)


C:\Users\Aditya Vardhan\AppData\Local\Temp\ipykernel_3528\3558881001.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("precision")
C:\Users\Aditya Vardhan\AppData\Roaming\Python\Python312\site-packages\datasets\load.py:759: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Using the latest cached version of the module from C:\Users\Aditya Vardhan\.cache\huggingface\modules\datasets_modules\metrics

In [18]:
from transformers import ViTForImageClassification

# # labels = ds['train'].features['labels'].names

# model = ViTForImageClassification.from_pretrained(
#     model_name_or_path,
#     num_labels=2,
# )


In [19]:
# model.save_pretrained("vitpre")

In [20]:
# Define the path from where to load the model
model_directory = "vitpre"

# Load the model from the saved directory
model = ViTForImageClassification.from_pretrained(model_directory)

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  save_steps=10,
  eval_steps=10,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=3,
  remove_unused_columns=False,
  push_to_hub=False,
  # report_to='tensorboard',
  load_best_model_at_end=True,
)


C:\Users\Aditya Vardhan\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,  # Number of evaluation steps to wait before stopping
    early_stopping_threshold=0.01,  # Minimum change to qualify as an improvement
)

In [23]:
from transformers import Trainer

def custom_processor(images):
    return processor(images, do_normalize = False, return_tensors='pt')

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    # train_dataset=prepared_ds["train"]                 ,
    train_dataset = train_dataset,
    # eval_dataset=prepared_ds["validation"],
    eval_dataset = validation_dataset,
    # tokenizer = custom_processor
    callbacks = [early_stopping_callback]
)


In [24]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()


Step,Training Loss,Validation Loss,Precision
10,0.467500,0.146019,0.976744
20,0.218500,0.188655,0.926471
30,0.131600,0.061196,1.000000
40,0.081000,0.265167,1.000000
50,0.099500,0.141773,0.968000
60,0.206900,0.037538,0.984375
70,0.037900,0.078431,0.954198
80,0.109800,0.113450,0.926471
90,0.095800,0.123448,0.913043
100,0.076200,0.072103,1.000000


***** train metrics *****
  epoch                    =      2.2535
  total_flos               = 184466620GF
  train_loss               =      0.1075
  train_runtime            =  1:32:38.76
  train_samples_per_second =        4.08
  train_steps_per_second   =       0.255


In [25]:
metrics = trainer.evaluate(validation_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)


***** eval metrics *****
  epoch                   =     2.2535
  eval_loss               =     0.0113
  eval_precision          =        1.0
  eval_runtime            = 0:02:00.21
  eval_samples_per_second =      2.096
  eval_steps_per_second   =      0.133
